# Finding the Safest Walking Paths 🗺️

## What Are We Building?

Imagine you're walking home and Google Maps shows you 5 different routes. This system helps you find the **3 safest routes** by checking:
- **Crimes and incidents** nearby
- **Potential urban and social risks** nearby

Then it ranks all your routes from safest to most risky.


## How It Works (Simple Workflow)

```
1. User inputs a Starting Point and Destination location
2. Application generates 10 routes from Graphhopper API
3. Rank 10 routes based on risk scoring algorithm (higher risk score = less safe), pulling from DataSF API endpoints at a frequency of every 10 minutes
4. We give you back the 3 routes with LOWEST danger scores
```

The lower the score, the safer the route! 🎯


## 📚 Some Important Terms (Don't Worry, We'll Explain!)

Before we start, let's learn a few simple words:

**Polypath** = A walking route (like a list of "turn here, then here")  
**API** = A way to get data from websites (like asking a question and getting an answer)  
**Risk Score** = How dangerous a route is (higher number = more dangerous)

**The Magic Formula:** if w=3, f(d,t,w) = (ReLU(w-w(t/72)))² × e^(-(d)²/0.02), else f(d,t,w) = (ReLU(w-w(t/24)))² × e^(-(d)²/0.02)
- Don't worry about this yet! We'll explain it step by step.
- **d** = distance (things close matter more than far things)
- **t** = time since incident (recent incidents matter more than old ones)
- **ReLU** = makes sure values never go negative
- **w** = risk score of singular event (1 = least dangerous, 2 = medium danger, 3 = most dangerous)
- Just know: "recent and close incidents are more dangerous"


## Step 1: Getting the Tools We Need

Think of this like buying ingredients for a recipe. We need to install some "tools" (called libraries) first.

**What's a library?**  
It's like a toolbox with pre-made tools. Instead of building a hammer from scratch, you just get one from the toolbox!

Run the cell below to install everything:


In [1]:
# This installs the tools we need
# Just like installing apps on your phone!
%pip install boto3 requests numpy pandas geopy --quiet

print("✅ Tools installed! We're ready to start!")


Note: you may need to restart the kernel to use updated packages.
✅ Tools installed! We're ready to start!


## Step 2: Bringing in Our Tools

Now we're going to "import" our tools. Think of it like opening your toolbox.

**Analogy:** Imagine you're cooking:
- You need a knife → you "import" it from the kitchen
- You need a cutting board → you "import" it from the cupboard

Here, we're importing Python tools from libraries!


In [2]:
# Import = bring in the tools
import boto3        # This talks to Amazon Web Services (AWS)
import requests     # This gets data from the internet
import json         # This reads/writes data in a special format
import numpy as np  # This does math calculations
import pandas as pd  # This organizes data in tables
from geopy.distance import geodesic  # This calculates distances on Earth
from typing import List, Dict, Tuple  # This just helps us write cleaner code

print("✅ All tools loaded! We're ready to work!")
print("\n💡 Tip: The 'import' command loads pre-written code so we don't have to write everything ourselves!")


✅ All tools loaded! We're ready to work!

💡 Tip: The 'import' command loads pre-written code so we don't have to write everything ourselves!


## Step 3: Setting Up AWS (Amazon Web Services)

**What is AWS?**  
Think of AWS as Amazon's cloud services - it's like renting storage space and smart computers online.

**Why do we need it?**
1. **S3** = We store our data here (like a filing cabinet)
2. **Bedrock** = We use AI to understand how dangerous incidents are

### 📝 You Need to Set This Up (Don't Worry, It's Easy!)

1. Go to https://aws.amazon.com
2. Create a free account
3. Go to S3 (looks like a storage symbol)
4. Create a bucket (just click "Create bucket" and name it)
5. Get your credentials (Access Key ID and Secret Key)

**Important:** Never share your credentials! Keep them secret like a password.

### 🔧 Configuration

Fill in your details below:


In [3]:
# ⚠️ IMPORTANT: Fill in your AWS details here
# You'll get these from your AWS account

# What region are you in? (us-east-1 is a common choice for USA)
AWS_REGION = "us-east-1"  # 🌍 Change this to your region

# What's your S3 bucket name? (This is where we store data)
S3_BUCKET_NAME = "my-path-risk-data"  # 📦 Change this to your bucket name

# Now we create the "clients" (think of them as assistants)
# They'll help us talk to AWS
s3_client = boto3.client('s3', region_name=AWS_REGION)
bedrock_client = boto3.client('bedrock-runtime', region_name=AWS_REGION)

print(f"✅ Connected to AWS in region: {AWS_REGION}")
print("💡 If this fails, make sure you've set up your AWS credentials!")


✅ Connected to AWS in region: us-east-1
💡 If this fails, make sure you've set up your AWS credentials!


## Step 4: Understanding the Data We'll Get

Before we write code, let's understand what data we're getting.

### Data from Crime API

When police respond to a call, they record it. We get this data like:
```json
  {
    "entry_datetime": "2025-10-26T02:11:01.000",
    "call_type_original_desc": "FIGHT NO WEAPON",
    "call_type_final_desc": "FIGHT NO WEAPON",
    "intersection_name": "01ST ST \\ HOWARD ST",
    "intersection_point": {
      "type": "Point",
      "coordinates": [-122.396034766, 37.788535931]
    }
  }

```

### Data from 311 API

When someone reports a street problem (like a pothole), we get data like:
```json
  {
    "requested_datetime": "2025-10-24T22:49:55.000",
    "status_description": "Open",
    "service_name": "Encampment",
    "service_subtype": "encampment",
    "point_geom": {
      "type": "Point",
      "coordinates": [-122.42352075, 37.78433716]
    }
  }

```

**In simple terms:** We're getting lists of "bad things that happened" with their locations!


## Step 5: Getting Crime Data from the Internet

**What is an API?**  
Think of it like this: You want to know the weather. You ask Google "What's the weather?" and Google gives you the answer. That's how an API works - you ask a question, you get an answer!

**How it works:**
1. We ask the Crime API: "Give me all crimes in this area"
2. It returns a list of crimes
3. We use that list!

Let's write the function:


In [ ]:
def fetch_crime_data(area_bounds, api_key=None):
    """
    Get crime data from SF Open Data API.
    
    This is like asking a website: "What crimes happened here?"
    
    Input:
    - area_bounds: Where we want to look (like GPS coordinates of an area)
    - api_key: Not needed for SF Open Data (they allow public access)
    
    Output:
    - A list of crimes that happened in that area
    """
    
    # SF Open Data API endpoint with specific crime types
    api_url = "https://data.sfgov.org/resource/gnap-fj3t.json?$query=SELECT%0A%20%20%60entry_datetime%60%20AS%20%60entry_datetime%60%2C%0A%20%20%60call_type_original_desc%60%20AS%20%60call_type_original_desc%60%2C%0A%20%20%60call_type_final_desc%60%20AS%20%60call_type_final_desc%60%2C%0A%20%20%60intersection_name%60%20AS%20%60intersection_name%60%2C%0A%20%20%60intersection_point%60%20AS%20%60intersection_point%60%0AWHERE%0A%20%20caseless_one_of(%0A%20%20%20%20%60call_type_original_desc%60%2C%0A%20%20%20%20%22EXPLOSIVE%20FOUND%22%2C%0A%20%20%20%20%22SUSPICIOUS%20PERSON%22%2C%0A%20%20%20%20%22FIGHT%20W%2FWEAPONS%22%2C%0A%20%20%20%20%22FIGHT%20NO%20WEAPON%22%2C%0A%20%20%20%20%22ASSAULT%20%2F%20BATTERY%20DV%22%2C%0A%20%20%20%20%22PURSE%20SNATCH%22%2C%0A%20%20%20%20%22EXPLOSION%22%2C%0A%20%20%20%20%22ROBBERY%22%2C%0A%20%20%20%20%22THREATS%20%2F%20HARASSMENT%22%2C%0A%20%20%20%20%22STRONGARM%20ROBBERY%22%2C%0A%20%20%20%20%22INDECENT%20EXPOSURE%22%2C%0A%20%20%20%20%22PERSON%20BREAKING%20IN%22%2C%0A%20%20%20%20%22BURGLARY%22%0A%20%20)%0AORDER%20BY%20%60entry_datetime%60%20DESC%20NULL%20LAST"
    
    # Query for specific crime types
    query = {
        "$query": """SELECT
  entry_datetime,
  call_type_original_desc,
  call_type_final_desc,
  intersection_name,
  intersection_point
WHERE
  caseless_one_of(
    call_type_original_desc,
    "EXPLOSIVE FOUND",
    "SUSPICIOUS PERSON",
    "FIGHT W/WEAPONS",
    "FIGHT NO WEAPON",
    "ASSAULT / BATTERY DV",
    "PURSE SNATCH",
    "EXPLOSION",
    "ROBBERY",
    "THREATS / HARASSMENT",
    "STRONGARM ROBBERY",
    "INDECENT EXPOSURE",
    "PERSON BREAKING IN",
    "BURGLARY"
  )
ORDER BY entry_datetime DESC NULL LAST""",
        "$limit": 500  # Limit results
    }
    
    try:
        # Ask the website for data
        response = requests.get(api_url, params=query)
        
        # Check if we got good data back
        response.raise_for_status()
        
        # Convert the response to something we can use
        crimes = response.json()
        
        print(f"✅ Got {len(crimes)} crime incidents from SF Open Data!")
        return crimes
        
    except Exception as e:
        # If something went wrong, tell us
        print(f"❌ Error: {e}")
        return []

print("✅ Function created! This function gets crime data from SF Open Data.")
print("\n💡 This queries SF Open Data API for the specific crime types we need!")


✅ Function created! This function gets crime data from SF Open Data.

💡 This queries SF Open Data API for the specific crime types we need!


## Step 6: Getting 311 Data (Street Problems)

**What is 311?**  
311 is a city service people call to report problems like:
- Aggressive/threatening behavior
- Homeless encampments
- And more

We want this data because these things can pose a threat to pedestrians in many cases!



In [ ]:
def fetch_311_data(area_bounds, api_key=None):
    """
    Get 311 data from SF Open Data API.
    
    This asks: "What street problems (Encampments, Aggressive/Threatening) were reported here?"
    
    Input:
    - area_bounds: Where to look (not used in this implementation, but kept for compatibility)
    - api_key: Not needed for SF Open Data
    
    Output:
    - List of 311 incidents
    """
    
    # SF Open Data 311 API endpoint
    api_url = "https://data.sfgov.org/resource/vw6y-z8j6.json?$query=SELECT%0A%20%20%60requested_datetime%60%2C%0A%20%20%60status_description%60%2C%0A%20%20%60service_name%60%2C%0A%20%20%60service_subtype%60%2C%0A%20%20%60point_geom%60%0AWHERE%0A%20%20caseless_one_of(%0A%20%20%20%20%60service_name%60%2C%0A%20%20%20%20%22Aggressive%2FThreatening%22%2C%0A%20%20%20%20%22Encampment%22%2C%0A%20%20%20%20%22Encampments%22%0A%20%20)%0A%20%20AND%20(%60requested_datetime%60%0A%20%20%20%20%20%20%20%20%20%3E%20%222025-10-19T01%3A26%3A14%22%20%3A%3A%20floating_timestamp)%0AORDER%20BY%20%60requested_datetime%60%20DESC%20NULL%20LAST"
    
    # Query for Aggressive/Threatening and Encampment incidents
    query = {
        "$query": """SELECT
  requested_datetime,
  status_description,
  service_name,
  service_subtype,
  point_geom
WHERE
  caseless_one_of(
    service_name,
    "Aggressive/Threatening",
    "Encampment",
    "Encampments"
  )
ORDER BY requested_datetime DESC NULL LAST""",
        "$limit": 500  # Limit results
    }
    
    try:
        # Ask for the data
        response = requests.get(api_url, params=query)
        response.raise_for_status()
        
        # Get the data
        incidents = response.json()
        
        print(f"✅ Got {len(incidents)} 311 incidents from SF Open Data!")
        return incidents
        
    except Exception as e:
        print(f"❌ Error: {e}")
        return []

print("✅ Function created! This gets Encampment and Aggressive/Threatening reports.")
print("\n💡 This queries SF Open Data API for encampments and threatening behavior!")


✅ Function created! This gets Encampment and Aggressive/Threatening reports.

💡 This queries SF Open Data API for encampments and threatening behavior!


## Step 8: Understanding Risk Levels 🤔

This is important! We need to decide: **How dangerous is each incident?**

We use three levels with **specific crime types**:

- **3 (High Risk)** = Very dangerous crimes:
  - EXPLOSIVE FOUND, EXPLOSION
  - ROBBERY, STRONGARM ROBBERY
  - ASSAULT, BATTERY

- **2 (Medium Risk)** = Somewhat dangerous crimes:
  - PURSE SNATCH, INDECENT EXPOSURE
  - FIGHT W/WEAPONS, FIGHT NO WEAPONS
  - PERSON BREAKING IN

- **1 (Low Risk)** = Less serious crimes:
  - SUSPICIOUS PERSON
  - THREATS / HARRASMENT, Aggressive/Threatening
  - Encampment
  - Encampments

**Special case: ENCAMPMENT(S)** 🏕️
- If status is "Closed" → doesn't get added to map / gets removed if updating an old one
- If status is open → doesn't decay with time (stays dangerous until closed)

**Why assign risk?**  
Not all problems are equally dangerous. An actual robbery or assault is worse than just a suspicious person!


In [ ]:
def assign_risk(incident, incident_type):
    """
    Decide how dangerous an incident is.
    
    This is like judging: "How bad is this problem?"
    
    Input:
    - incident: Details about what happened
    - incident_type: Is this crime data or 311 data?
    
    Output:
    - A number (1, 2, or 3) saying how dangerous it is
    """
    
    if incident_type == "crime":
        # Get the crime description
        description = incident.get("call_type_original_desc", "").upper()
        
        # High risk crimes (w=3): Very dangerous
        high_risk_crimes = ["EXPLOSIVE FOUND", "EXPLOSION", "ROBBERY", "STRONGARM ROBBERY", "ASSAULT", "BATTERY"]
        if any(crime in description for crime in high_risk_crimes):
            return 3  # 🚨 Very dangerous!
        
        # Medium risk crimes (w=2): Somewhat dangerous
        medium_risk_crimes = ["PURSE SNATCH", "INDECENT EXPOSURE", "FIGHT W/WEAPONS", "FIGHT NO WEAPONS", "PERSON BREAKING IN"]
        if any(crime in description for crime in medium_risk_crimes):
            return 2  # ⚠️ Somewhat dangerous
        
        # Low risk crimes (w=1): Not very dangerous
        low_risk_crimes = ["SUSPICIOUS PERSON", "THREATS/HARASSMENT"]
        if any(crime in description for crime in low_risk_crimes):
            return 1  # ✅ Not very dangerous
        
        return 0
    
    else:  # It's 311 data
        # SF 311 data uses service_name
        service_name = incident.get("service_name", "").upper()
        
        # Check for specific service types
        if "Encampment" or "Encampments" or "Aggressive/Threatening" in service_name:
            return 1  # Low risk, but persistent
        
        


🧪 Testing risk assignment:
EXPLOSIVE FOUND → Risk: 3
ROBBERY → Risk: 3
PURSE SNATCH → Risk: 2
ENCAMPMENT → Risk: 1
SIDEWALK HAZARD → Risk: 3


## Step 9: Measuring Distance 🏃

**What we're doing:**  
We need to know: "How far is this incident from our walking route?"

**Why?**  
An incident 10 feet away is MUCH more relevant than one 2 miles away!

**The Math (Don't Worry!):**  
We use "geodesic distance" - this is fancy talk for "distance on Earth" (not as a bird flies, but walking distance).

**Real Life Example:**  
If there's an assault 100 meters from your route, that's concerning.  
If there's an assault 5 kilometers away, who cares? You won't walk near it.

Let's write a function to measure this!


In [9]:
def min_distance_to_path(incident_coords, path):
    """
    Find the shortest distance from an incident to any point on the path.
    
    Think of it like this: 
    If an incident happened at point A, and your route goes through points B, C, D...
    What's the shortest distance from A to any of those points?
    
    Input:
    - incident_coords: Where the incident happened (latitude, longitude)
    - path: Your route (a list of GPS coordinates)
    
    Output:
    - The distance in kilometers
    """
    
    # Safety check: Make sure we have a real path
    if len(path) < 2:
        return float('inf')  # If no path, assume it's infinitely far away
    
    # Start with a really big number
    min_dist = float('inf')
    
    # Check distance to every point on the path
    for path_point in path:
        try:
            # Calculate distance (using geodesic = Earth's curve)
            dist = geodesic(incident_coords, path_point).meters / 1000
            # ^convert meters to kilometers
            
            # Keep track of the smallest distance we found
            min_dist = min(min_dist, dist)
        except:
            # If something goes wrong, just skip it
            continue
    
    return min_dist

# Let's test it!
test_path = [(37.7849, -122.4094), (37.7855, -122.4086)]
test_incident = (37.7865, -122.4080)
test_distance = min_distance_to_path(test_incident, test_path)

print(f"🧪 Test:")
print(f"Incident at: {test_incident}")
print(f"Path goes through: {test_path}")
print(f"Distance: {test_distance:.4f} km")
print("\n💡 The smaller the number, the closer the incident is to your route!")


🧪 Test:
Incident at: (37.7865, -122.408)
Path goes through: [(37.7849, -122.4094), (37.7855, -122.4086)]
Distance: 0.1229 km

💡 The smaller the number, the closer the incident is to your route!


## Step 10: THE MAGIC FORMULA 🎩✨

This is where the magic happens! This is the core of our system.

### The Formula (With Time Decay!): 
**For high risk (w=3):** f(d,t,w) = (ReLU(3-3t/72))² × e^(-d²/0.02)  
**For lower risk (w=1,2):** f(d,t,w) = (ReLU(w-wt/24))² × e^(-d²/0.02)

**Breaking it down in simple terms:**

#### Part 1: The Risk Value (with Time!)
- w = How dangerous the incident is (1, 2, or 3)
- t = Time since incident (in hours)
- **High-risk incidents (w=3):** Take 72 hours to fully decay (3 days!)
- **Lower-risk incidents (w=1,2):** Take 24 hours to fully decay (1 day!)

**Why different times?**  
Serious crimes stay dangerous for longer!

#### Part 2: ReLU Function
ReLU(x) = max(0, x) - Returns 0 if negative, otherwise returns the value itself.

**What it does:**  
Prevents the risk from going negative (makes sense - you can't have "negative danger"!)

#### Part 3: e^(-d²/0.02) (Distance Decay)
- d = Distance in kilometers
- Makes things further away matter less

**How it works:**
- Close incident (0.1 km): Contributes a lot
- Medium distance (0.5 km): Contributes some  
- Far away (1+ km): Contributes almost nothing

#### Putting It Together:
A **recent, high-risk incident close** to your path has a huge impact.  
An **old, low-risk incident far away** has almost no impact.

**That's the whole point!** 🎯


In [10]:
def calculate_risk_score(path, incidents, incident_type, current_time=None):
    """
    Calculate the total danger score for a path.
    
    How it works:
    1. For each incident near the path
    2. Get its risk level (1, 2, or 3)
    3. Measure how far it is from the path
    4. Calculate time since incident
    5. Apply the magic formula with time decay
    6. Add it all up
    
    Special case: ENCAMPMENT
    - If status is CLOSED → skip (contributes 0 to risk)
    - If status is open → doesn't decay with time
    
    Input:
    - path: Your route (list of GPS points)
    - incidents: All the bad things that happened
    - incident_type: Is this crime or 311 data?
    - current_time: Current timestamp (for time calculation)
    
    Output:
    - Total danger score (lower is safer!)
    """
    
    total_risk = 0.0
    
    # Look at each incident
    for incident in incidents:
        
        # Step 1: Get the location of this incident
        if incident_type == "crime":
            # SF Crime data has location as [longitude, latitude] in intersection_point
            coords = incident.get("intersection_point", {}).get("coordinates", [])
            if len(coords) < 2:
                continue  # Skip if we don't have a location
            incident_coords = (coords[1], coords[0])  # Convert to (lat, lon)
        else:  # It's 311 data
            # SF 311 data has point_geom in GeoJSON format: {"type": "Point", "coordinates": [lon, lat]}
            point_geom = incident.get("point_geom")
            if point_geom:
                coords = point_geom.get("coordinates", [])
                if len(coords) >= 2:
                    incident_coords = (coords[1], coords[0])  # Convert to (lat, lon)
                else:
                    continue  # Skip if invalid
            else:
                # Fallback for old format
                lat = incident.get("Latitude")
                lon = incident.get("Longitude")
                if lat is None or lon is None:
                    continue  # Skip if missing
                incident_coords = (lat, lon)
        
        # Step 2: Get the risk level (1, 2, or 3)
        w = assign_risk(incident, incident_type)
        
        # Step 3: Find distance to path
        d = min_distance_to_path(incident_coords, path)
        
        # SPECIAL CASE: Check if this is ENCAMPMENT
        if incident_type == "crime":
            description = incident.get("call_type_original_desc", "").upper()
        else:
            # For 311 data, check service_name
            description = incident.get("service_name", "").upper()
        is_encampment = "ENCAMPMENT" in description
        
        # If it's ENCAMPMENT and status is CLOSED, skip it (contributes 0)
        if is_encampment:
            # Check status (for 311 it's status_description, for crimes it's status)
            if incident_type == "crime":
                status = incident.get("status", "").upper()
            else:
                status = incident.get("status_description", "").upper()
            if status == "CLOSED":
                continue  # Skip this incident, don't contribute to risk
        
        # Step 4: Calculate time since incident
        # Get incident time from data
        if incident_type == "crime":
            incident_time_str = incident.get("entry_datetime")
        else:
            # For SF 311 data, it's requested_datetime
            incident_time_str = incident.get("requested_datetime") or incident.get("timestamp")
        
        # Calculate time difference in hours
        t = 0  # Default: assume recent incident (0 hours ago)
        if current_time and incident_time_str:
            try:
                from datetime import datetime
                # Parse the time string
                incident_time = datetime.fromisoformat(incident_time_str.replace('Z', '+00:00'))
                if isinstance(current_time, str):
                    current_time = datetime.fromisoformat(current_time.replace('Z', '+00:00'))
                # Calculate difference in hours
                time_diff = current_time - incident_time
                t = time_diff.total_seconds() / 3600  # Convert to hours
            except:
                t = 0  # If we can't parse, assume recent
        
        # Step 5: Apply the magic formula WITH TIME DECAY!
        # ReLU function: max(0, x)
        if is_encampment:
            # ENCAMPMENT doesn't decay with time - use full risk value
            relu_term = w
        elif w == 3:
            # High-risk incidents: decay over 72 hours
            relu_term = max(0, 3 - 3*t/72)
        else:
            # Lower-risk incidents: decay over 24 hours
            relu_term = max(0, w - w*t/24)
        
        risk_value = (relu_term ** 2) * np.exp(-(d ** 2) / 0.02)
        
        # Step 6: Add to total
        total_risk += risk_value
    
    return total_risk

print("✅ Function created! This calculates how dangerous your path is.")
print("\n📊 Lower score = Safer path!")
print("💡 Example: Path A = 5.2, Path B = 12.7 → Path A is safer!")
print("\n🎯 Key feature: Recent incidents matter MORE than old ones!")


✅ Function created! This calculates how dangerous your path is.

📊 Lower score = Safer path!
💡 Example: Path A = 5.2, Path B = 12.7 → Path A is safer!

🎯 Key feature: Recent incidents matter MORE than old ones!


## Step 11: THE MAIN FUNCTION 🎪

This brings everything together! Think of it as the conductor of an orchestra - it makes sure everything happens in the right order.

**What it does:**
1. Users inputs starting point and destination
2. 10 best/most efficient routes are returned (before filter of safety)
3. Get crime & 311 cases data
4. For each path, calculate risk score -- adding up individual spots' risk added to the path based on the algorithm/formula to aggregate a risk score total for each path
5. Sort paths by risk
6. Return the 3 safest paths

Let's build it!


In [11]:
def find_lowest_risk_paths(polypaths, area_bounds=None, crime_api_key=None, api_311_key=None):
    """
    THE BIG ONE! This does everything!
    
    Input:
    - polypaths: List of possible routes
    - area_bounds: Not used for SF Open Data (optional)
    - crime_api_key: Not needed for SF Open Data (optional)
    - api_311_key: Not needed for SF Open Data (optional)
    
    Output:
    - Top 3 safest paths
    """
    
    print(f"🔍 Analyzing {len(polypaths)} paths...")
    print("=" * 60)
    
    # STEP 1: Get the data from SF Open Data
    print("\n📞 Fetching crime data from SF Open Data...")
    crime_data = fetch_crime_data(area_bounds, crime_api_key)
    
    print("\n📞 Fetching 311 data from SF Open Data...")
    incidents_311 = fetch_311_data(area_bounds, api_311_key)
    
    # STEP 2: Save to S3 (backup!)
    print("\n💾 Saving to cloud storage...")
    save_to_s3({"crimes": crime_data}, S3_BUCKET_NAME, "crime_data.json")
    save_to_s3({"incidents": incidents_311}, S3_BUCKET_NAME, "311_data.json")
    
    # STEP 3: Calculate risk for each path
    path_risks = []
    
    for i, path in enumerate(polypaths):
        print(f"\n📊 Analyzing Path {i+1}...")
        
        # Get current time for time-based calculations
        from datetime import datetime
        current_time = datetime.now().isoformat()
        
        # Calculate risk from crimes (with time!)
        crime_risk = calculate_risk_score(path, crime_data, "crime", current_time)
        
        # Calculate risk from 311 incidents (with time!)
        incident_risk = calculate_risk_score(path, incidents_311, "311", current_time)
        
        # Total risk = crime risk + 311 risk
        total_risk = crime_risk + incident_risk
        
        path_risks.append((path, total_risk))
        print(f"   Total Risk Score: {total_risk:.4f}")
    
    # STEP 4: Sort by risk (lowest first = safest)
    path_risks.sort(key=lambda x: x[1])
    
    # STEP 5: Get top 3
    top_3_paths = [path for path, risk in path_risks[:3]]
    
    print("\n" + "=" * 60)
    print("🏆 RESULTS: Top 3 Safest Paths")
    print("=" * 60)
    for i, (path, risk) in enumerate(path_risks[:3]):
        print(f"\n{i+1}. Path {i+1} - Risk Score: {risk:.4f} (SAFEST!)" if i == 0 else f"\n{i+1}. Path {i+1} - Risk Score: {risk:.4f}")
    
    return top_3_paths

print("✅ Main function created!")
print("\n🎉 This is the function you'll actually use!")


✅ Main function created!

🎉 This is the function you'll actually use!


## 🎓 What You Learned!

Let's review what we built:

### ✅ The System
1. **Gets data** from Crime APIs and 311 APIs
2. **Assigns risk** to each incident (1, 2, or 3)
3. **Measures distance** from incidents to paths
4. **Calculates risk** using the magic formula
5. **Returns** the 3 safest paths

### 🧮 The Math
**Formula (with time decay!):** 
- **For w=3:** f(d,t,w) = (ReLU(3-3t/72))² × e^(-d²/0.02)
- **For w=1,2:** f(d,t,w) = (ReLU(w-wt/24))² × e^(-d²/0.02)
- **w** = Risk value (1, 2, or 3)
- **d** = Distance in kilometers
- **Lower total** = Safer path! 🎯


Happy analyzing! 🗺️✨
